# BackEnd for Vacuum Gauge Explorer
Contains the classes needed to operate the Vacuum Gauge explorer:
* **VGcontext**: Contains information at the level of a fill
    * Fill Number
    * Beam Energy
    * Beam Intensity
    * Bunch Lenghts
    * Probe Catalogue (file containing names of valid probes)
* **VGprobe**: Contains information at the level of a probe e.g. raw readings
    * Probe ID 
    * Time Readings
    * Pressure Readings
    * Processed Readings
* **VGplot**: Contains methods needed to visualize 1 or more probes
    * MultiPlot (given VGcontext)
    * SinglePlot (given VGprobe)
* **VGcontrol**: Contrains the tools for stepping through gauges intelligently and applying analyzers to them 

In [10]:
%run BackEnd_Plotters.ipynb
%run BackEnd_DataProcessing.ipynb
import re
import pytimber
import os
import pandas as pd
import pickle
import copy
from IPython.display import clear_output
db = pytimber.LoggingDB()

/usr/local/lib/swan/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Populating the interactive namespace from numpy and matplotlib


In [15]:
class VGprobe(object):
    def __init__(self, probe_id, fill_no, verbose = False):
        self.verbose = verbose
        self.probe_is_valid = True
        self.fill_no = fill_no
        if type(VGcontext) == VGcontext:
            self.context = VGcontext
        else:
            self.probe_is_valid = False
            return
        
        file_name = "Probe_{0}_Fill{1}.p".format(gauge_id,str(self.fill_no))
        folder = os.path.join(os.getcwd(),"data","probes",str(self.fill_no),gauge_id) 
        file_path = os.path.join(folder,file_name)
        if os.path.isfile(file_path):
            if verbose: print("Loading existing data for {} in Fill {}".format(gauge_id,fill_no))
            with open(file_path,"rb") as pgd_file:
                pgd = pickle.load(pgd_file)
            # Copy attributes from Processed Gauge Data (pgd) to Probe Object
            self.__dict__.update(pgd.__dict__)
        else:
            if verbose: print("Saving new data for {} in Fill {}".format(gauge_id,fill_no))
            try:
                pgd = processed_gauge_data(gauge_id, row['Fill'])
                pgd.generate_data()
            except:
                self.probe_is_valid = False
                return
            with open(file_path,"wb") as pgd_file:
                pickle.dump(pgd, pgd_file)

In [2]:
class ContextError(Exception):
    pass

In [6]:
class VGcontext(object):
    def pick_fill_number(self,fillNo=None):
        if fillNo is None:
            try:
                maxFillNo = db.getLHCFillData(None)['fillNumber']
            except:
                maxFillNo = 7494
            print("Valid Fill Numbers: 0 - %d (inclusive)" % (maxFillNo))
            fillNo = input("Please select a fill number >>>")
            while(True):
                try:
                    fillNo = int(fillNo)
                    if(fillNo < 0 or fillNo > maxFillNo):
                        print("Please enter a fill number in the correct range")
                        input("Select a fill number >>>")
                        continue
                    break
                except:
                    print("Please enter a valid integer")
                    input("Select a fill number >>>")
        return fillNo
    
    def __init__(self, fillNo=None, verbose = False):
        self.verbose = verbose
        self.fillNo = fillNo if fillNo is not None else self.pick_fill_number()
        self.data_is_valid = True

        fill = db.getLHCFillData(self.fillNo)
        if fill is None:
            self.data_is_valid = False
            #"Could not retrieve fill data for fill {} through PyTimber".format(self.fillNo)
        self.t1 = fill['startTime']
        self.t2 = fill['endTime']
        date1 = datetime.datetime.fromtimestamp(self.t1)
        date2 = datetime.datetime.fromtimestamp(self.t2)
        timeBetween = date2 - date1
        if timeBetween.seconds<(60*60*5):
            self.data_is_valid = False
            self.msg = "{} is too short at {} to be loaded".format(self.fillNo,timeBetween)

        self.beamNumber=1
        beamEnergyVariable='LHC.BOFSU:OFSU_ENERGY'
        beamEnergy=db.get(beamEnergyVariable, self.t1, self.t2)
        if not all(beamEnergy): #True == Non-zeros present
            self.data_is_valid = False
            self.msg = "Beam Energy in Fill {} is INCOMPLETE".format(self.fillNo)
        self.beamEnergy = beamEnergy[beamEnergyVariable]

        beamIntensityVariable= 'LHC.BCTFR.A6R4.B'+str(self.beamNumber)+':BEAM_INTENSITY'
        beamIntensity=db.get(beamIntensityVariable, self.t1, self.t2)
        if not all(beamIntensity): #True == Non-zeros present4
            self.data_is_valid = False
            self.msg = "Beam Intensity in Fill {} is INCOMPLETE".format(self.fillNo)
        self.beamIntensity = beamIntensity[beamIntensityVariable]

        beamBunchVariable='LHC.BQM.B'+str(self.beamNumber)+':BUNCH_LENGTH_MEAN'
        beamBunchLengths=db.get(beamBunchVariable, self.t1, self.t2)
        if not all(beamBunchLengths): #True == Non-zeros present
            self.data_is_valid = False
            self.msg = "Bunch Lengths in Fill {} are INCOMPLETE".format(self.fillNo)
        self.beamBunchLengths = beamBunchLengths[beamBunchVariable]
        
        folder = os.path.join(os.getcwd(),"data","contexts",str(self.fillNo))
        if not os.path.exists(folder):
            os.makedirs(folder)
        fileName = "{}_verified_probe_catalogue.csv".format(self.fillNo)
        
        if not os.path.isfile(os.path.join(folder,fileName)):
            self.probe_catalogue = pd.DataFrame(data=None,index=None,columns=["Probe ID","Status"]) #Empty
        else:
            self.probe_catalogue = pd.read_csv(os.path.join(folder,fileName))
        

In [4]:
class VGcontrol(object):
    def __init__(self, verbose = 0):
        self.verbose = verbose
        self.current_context = VGcontext(fillNo=None)
        try:
            self.maxFillNo = db.getLHCFillData(None)['fillNumber']
        except:
            self.maxFillNo = 7492
            
        # For feedback
        self.current_msg = ""
            
        # For Automatic Stepping
        self.bias = None
        self.teller = 0
        
        # For Fixing Locations
        self.saved_locations, self.save_locations = None, False
        
        # For classifier use
        self.try_to_classify=False
        
        self.main()
        
    def display_help_and_input(self, msg=None):
        info = """{0}{2}{1}
        Currently at: {3} of {4}
        {0}s:{1} reload current fill
        {0}d:{1} move forward one fill
        {0}a:{1} move backward one fill
        {0}j:{1} jump to a specific fill
        {0}b:{1} step through X fills in specified direction
        {0}f:{1} fix selected location until called again
        {0}x:{1} to quit this wizard
        {0}z:{1} analyze using Classifiers""".format(color.BOLD,color.END,msg,self.current_context.fillNo,self.maxFillNo)
        print(info)
        self.resp=input(">>>")
    

    def main(self):
        def move_forward_one_fill(self):
            if self.current_context.fillNo <= self.maxFillNo:
                self.current_context = VGcontext(fillNo=self.current_context.fillNo + 1)
            else:
                print("Reached furthest fill")

        def reload_current_fill(self):
            self.current_context = VGcontext(fillNo=self.current_context.fillNo) # Forces a reload

        def move_backward_one_fill(self):
            if self.current_context.fillNo >= 1:
                self.current_context = VGcontext(fillNo=self.current_context.fillNo - 1)
            else:
                 print("Reached earliest fill")

        def jump_to_specific_fill(self):
            self.current_context = VGcontext(fillNo=None)

        def step_through_X_fills(self): # NOTE: Should require you to fix locations before activating
            resp = input("Pick a direction, a: back, d: forward")
            bias_options = {"a":-1,"d":1}
            while self.bias is None:
                resp = resp.strip().lower()
                if resp in bias_options:
                    self.bias = bias_options[resp]
                else:
                    resp = input("Pick a direction, a: back, d: forward")
            resp = input("Type the number of fills to go through")

            while self.teller == 0:
                try:
                    resp = int(resp)
                except ValueError:
                    print("Please type a number")
                    resp = input("Type the number of fills to go through")
                    continue
                if (self.bias == 1 and resp < self.maxFillNo-self.current_context.fillNo) or (self.bias == -1 and resp < self.fillNo):
                    self.teller = resp
                    return
                else:
                    print("Chosen number would go out of bounds, try again")
                    resp = input("Type the number of fills to go through")

        def fix_locations(self):
            if self.saved_locations is not None:
                self.saved_locations, self.save_locations = None, False
            else:
                self.save_locations = True

        def apply_classifiers(self):
            self.try_to_classify=True
                        
        commands = {"a":move_backward_one_fill, "s":reload_current_fill, "d":move_forward_one_fill,
                    "j":jump_to_specific_fill, "b":step_through_X_fills, "f":fix_locations,
                    "z":apply_classifiers}
                        
        while True:
            if self.teller == 0:
                self.display_help_and_input(msg=self.current_msg)
            
            if self.bias is not None and self.teller > 0:
                self.current_context = VGcontext(fillNo=self.current_context.fillNo + self.bias)
                print(self.current_context.data_is_valid)
                if not self.current_context.data_is_valid:
                    continue
                else:
                    print("Reducing teller to {}".format(self.teller-1))
                    self.teller -= 1
            else:
                #clear_output()
                matches = re.findall("^\s*([a-zA-Z]){1}\s*$",self.resp)
                if len(matches) == 0:
                    self.current_msg = "{} is not a single letter, please try again".format(self.resp)
                    continue
                self.resp = matches[0].lower()
                if self.resp == "x":
                    break #EXIT THE APPLICATION
                elif self.resp in commands:
                    prevFillNo = self.current_context.fillNo
                    commands[self.resp](self)
                    if not self.current_context.data_is_valid:
                        self.bias = self.current_context.fillNo - prevFillNo
                        self.teller += 1
                        if self.verbose: print(self.current_context.msg)
                        continue
                           

In [5]:
VGcontrol(verbose=True)

Valid Fill Numbers: 0 - 7494 (inclusive)
Please select a fill number >>>5979

        Currently at: 5979 of 7494
        s: reload current fill
        d: move forward one fill
        a: move backward one fill
        j: jump to a specific fill
        b: step through X fills in specified direction
        f: fix selected location until called again
        x: to quit this wizard
        z: analyze using Classifiers
>>>d

        Currently at: 5980 of 7494
        s: reload current fill
        d: move forward one fill
        a: move backward one fill
        j: jump to a specific fill
        b: step through X fills in specified direction
        f: fix selected location until called again
        x: to quit this wizard
        z: analyze using Classifiers
>>>d
5981 is too short at 2:33:08.276000 to be loaded
True
Reducing teller to 0

        Currently at: 5982 of 7494
        s: reload current fill
        d: move forward one fill
        a: move backward one fill
        j: jump 

In [44]:
xx,yy = np.meshgrid(np.arange(1,35),np.arange(1,9))
location_matrix = np.core.defchararray.add(xx.astype(str),np.full((8,34),"R"))
location_matrix = pd.DataFrame(np.core.defchararray.add(zz,yy.astype(str)))
display(location_matrix)

# Formats
# 0:5 or 6 (twice)
# 2L% %R3 %L%       
resp = input(">>>")
while True:
    if re.match("\d+:\d+",resp):
        lower_lim,upper_lim = int(resp.split(":")[0]),int(resp.split(":")[1])
    elif re.match("\d+",resp):
        lim = int(resp)
    elif re.match("[\d|%][L|R][\d|%]",resp) and "%"in resp:
        choice = resp
display(location_matrix.iloc[0:5,5:10])


,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1L11,2L11,3L11,4L11,5L11,6L11,7L11,8L11,9L11,10L11,...,25L11,26L11,27L11,28L11,29L11,30L11,31L11,32L11,33L11,34L11
1,1L22,2L22,3L22,4L22,5L22,6L22,7L22,8L22,9L22,10L22,...,25L22,26L22,27L22,28L22,29L22,30L22,31L22,32L22,33L22,34L22
2,1L33,2L33,3L33,4L33,5L33,6L33,7L33,8L33,9L33,10L33,...,25L33,26L33,27L33,28L33,29L33,30L33,31L33,32L33,33L33,34L33
3,1L44,2L44,3L44,4L44,5L44,6L44,7L44,8L44,9L44,10L44,...,25L44,26L44,27L44,28L44,29L44,30L44,31L44,32L44,33L44,34L44
4,1L55,2L55,3L55,4L55,5L55,6L55,7L55,8L55,9L55,10L55,...,25L55,26L55,27L55,28L55,29L55,30L55,31L55,32L55,33L55,34L55
5,1L66,2L66,3L66,4L66,5L66,6L66,7L66,8L66,9L66,10L66,...,25L66,26L66,27L66,28L66,29L66,30L66,31L66,32L66,33L66,34L66
6,1L77,2L77,3L77,4L77,5L77,6L77,7L77,8L77,9L77,10L77,...,25L77,26L77,27L77,28L77,29L77,30L77,31L77,32L77,33L77,34L77
7,1L88,2L88,3L88,4L88,5L88,6L88,7L88,8L88,9L88,10L88,...,25L88,26L88,27L88,28L88,29L88,30L88,31L88,32L88,33L88,34L88


>>>0


,5,6,7,8,9
0,6L11,7L11,8L11,9L11,10L11
1,6L22,7L22,8L22,9L22,10L22
2,6L33,7L33,8L33,9L33,10L33
3,6L44,7L44,8L44,9L44,10L44
4,6L55,7L55,8L55,9L55,10L55
